# 03 draft value Notebook

This notebook implements the analysis for the 03 draft value stage of the Fantasy Football Analysis project.

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os

# Add the src directory to path
sys.path.append('..')

# Set display options
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 1000)

In [ ]:
# TODO: Implement 03 draft value analysis

In [ ]:
# Import project modules
from src.data.data_loader import load_config, load_all_data
from src.data.data_processor import load_csv_data
from src.analysis.draft_value import (
    analyze_adp_vs_performance,
    calculate_vbd_rankings
)
from src.visualization.draft_vis import (
    plot_round_performance,
    plot_position_value_by_round,
    plot_vbd_rankings
)
from src.utils.validation import validate_analysis_output

# Set pandas display options
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 1000)

In [ ]:
# 1. Load configuration and processed data
config = load_config()
performance_df = load_csv_data(os.path.join(config['data_paths']['processed_data'], 'player_performance.csv'))
vorp_df = load_csv_data(os.path.join(config['data_paths']['processed_data'], 'vorp_analysis.csv'))

In [ ]:
# 2. Analyze ADP vs. performance
adp_results = analyze_adp_vs_performance(performance_df, config['league']['teams'])

# Display round stats
print("Draft Round Performance Stats:")
display(adp_results['round_stats'])

# Display position-specific round stats
print("\nPosition-Specific Round Stats:")
display(adp_results['position_round_stats'].pivot_table(
    index='Draft_Round', 
    columns='Position', 
    values=['Avg_Points', 'Success_Rate', 'Bust_Rate']
).round(1))

# Display optimal positions by round
print("\nOptimal Positions by Round:")
display(adp_results['optimal_positions_by_round'])

In [ ]:
# 3. Calculate VBD rankings
vbd_df = calculate_vbd_rankings(vorp_df, config['analysis']['baseline_values'])

# Display top players by VBD
print("\nTop 20 Players by VBD:")
display(vbd_df.sort_values('VBD', ascending=False)[
    ['Player', 'FantPos', 'Team', 'Half_PPR', 'VBD', 'VBD_Rank', 'ADP']
].head(20))

# Display VBD by round and position if available
if 'vbd_by_round' in vbd_df.attrs:
    print("\nVBD by Round and Position:")
    vbd_by_round = vbd_df.attrs['vbd_by_round']
    display(vbd_by_round.pivot_table(
        index='Draft_Round', 
        columns='Position', 
        values='Avg_VBD'
    ).round(1))

In [ ]:
# 4. Create visualizations
# Round performance
plt.figure(figsize=(12, 8))
plot_round_performance(adp_results['round_stats'])
plt.tight_layout()
plt.savefig('../outputs/figures/round_performance.png')
plt.show()

# Position value by round
plt.figure(figsize=(14, 10))
plot_position_value_by_round(adp_results['position_round_stats'])
plt.tight_layout()
plt.savefig('../outputs/figures/position_value_by_round.png')
plt.show()

# VBD rankings
plt.figure(figsize=(12, 8))
plot_vbd_rankings(vbd_df)
plt.tight_layout()
plt.savefig('../outputs/figures/vbd_rankings.png')
plt.show()

In [ ]:
# 5. Save processed data
vbd_df.to_csv(os.path.join(config['data_paths']['processed_data'], 'vbd_rankings.csv'), index=False)

In [ ]:
# Save ADP analysis results
for key, df in adp_results.items():
    df.to_csv(os.path.join(config['data_paths']['processed_data'], f"{key}.csv"), index=False)

print("Draft value analysis completed!")